# 서울시 생활인구
## 서울 생활인구 현황 (2022.09.28. 기준)
### url: https://data.seoul.go.kr/dataVisual/seoul/seoulLivingPopulation.do
### 서울시 생활인구란? 
    - 서울시와 KT가 공공빅데이터와 통신데이터를 이용하여 추계한 서울의 특정지역, 특정시점에 존재하는 모든 인구

1. 집계구 단위 서울 생활인구(내국인)
    - url: https://data.seoul.go.kr/dataList/OA-14979/F/1/datasetView.do
    - 설명: 서울시가 보유한 공공데이터와 통신데이터로 측정한 특정시점에 서울의 특정 지역에 존재하는 인구 중 내국인

<!-- <br> -->


2. 집계구 단위 서울 생활인구(장기체류 외국인)
    - url: https://data.seoul.go.kr/dataList/OA-14978/F/1/datasetView.do
    - 설명:서울시가 보유한 공공데이터와 통신데이터로 측정한 특정시점에 서울의 특정 지역에 존재하는 인구 중 장기체류 외국인

<!-- <br> -->


3. 집계구 단위 서울 생활인구(단기체류 외국인)
    - url: https://data.seoul.go.kr/dataList/OA-14980/F/1/datasetView.do
    - 설명:서울시가 보유한 공공데이터와 통신데이터로 측정한 특정시점에 서울의 특정 지역에 존재하는 인구 중 단기체류 외국인

<!-- <br> -->



※ 개인정보 비 식별화를 위하여 ‘3명’ 이하인 경우 “ * ” 처리


In [1]:
import os
import glob
import pandas as pd
import numpy as np

In [2]:
from tqdm.auto import tqdm, trange
from time import sleep

## 1. 집계구 단위 서울 생활인구(내국인)
    - a)2022년 01월~08월
    - b)2019년, 2020년, 2021년
    - 2019년 10월 15-27일 데이터 없음
## 2. 집계구 단위 서울 생활인구(장기체류 외국인)
    - a)2022년 01월~08월
    - b)2019년, 2020년, 2021년
## 3. 집계구 단위 서울 생활인구(단기체류 외국인)
    - a)2022년 01월~08월
    - b)2019년, 2020년, 2021년

In [3]:
# 폴더 안 파일명 리스트 가져오기
local_people_2019_list = []
local_people_2020_list = []
local_people_2021_list = []

long_foreigner_list = []
temp_foreinger_list= []
pbar = tqdm(os.listdir(r"./data/집계구 단위 서울 생활인구/local_people/2019"))
for file in pbar:
    if file.endswith(".csv"):
        pbar.set_description(file)
        local_people_2019_list.append(file)
        
pbar = tqdm(os.listdir(r"./data/집계구 단위 서울 생활인구/local_people/2020"))
for file in pbar:
    if file.endswith(".csv"):
        pbar.set_description(file)
        local_people_2020_list.append(file)
        
pbar = tqdm(os.listdir(r"./data/집계구 단위 서울 생활인구/local_people/2021"))
for file in pbar:
    if file.endswith(".csv"):
        pbar.set_description(file)
        local_people_2021_list.append(file)

pbar = tqdm(os.listdir(r"./data/집계구 단위 서울 생활인구/long_foreigner/"))
for file in pbar:
    if file.endswith(".csv"):
        pbar.set_description(file)
        long_foreigner_list.append(file)
 
pbar = tqdm(os.listdir(r"./data/집계구 단위 서울 생활인구/temp_foreigner/"))
for file in pbar:
    if file.endswith(".csv"):
        pbar.set_description(file)
        temp_foreinger_list.append(file)

local_people_2019_list.sort()
local_people_2020_list.sort()
local_people_2021_list.sort()
long_foreigner_list.sort()
temp_foreinger_list.sort()

print("no. of local_people_2019 >>", len(local_people_2019_list))
print("no. of local_people_2020 >>", len(local_people_2020_list))
print("no. of local_people_2021 >>", len(local_people_2021_list))

print("no. of long_foreigner_list >>", len(long_foreigner_list))
print("no. of temp_foreinger_list >>", len(temp_foreinger_list))

  0%|          | 0/352 [00:00<?, ?it/s]

  0%|          | 0/366 [00:00<?, ?it/s]

  0%|          | 0/365 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

no. of local_people_2019 >> 351
no. of local_people_2020 >> 366
no. of local_people_2021 >> 365
no. of long_foreigner_list >> 1
no. of temp_foreinger_list >> 1


In [4]:
def data_process(file_list, path):
    df_sum_per_day_집계구_list = []
    df_sum_per_day_행정동_list = []
    problem_list = []
    pbar = tqdm(file_list)
    for file in pbar:
        pbar.set_description(file) # pbar description에 작업중인 파일명 업데이트
        try:df_temp = pd.read_csv(path + file, encoding="utf-8", na_values="*")
        except UnicodeDecodeError:df_temp = pd.read_csv(path + file, encoding="euc-kr", na_values="*")
        except Exception as e: 
            # df_temp = pd.read_csv(path + file, encoding="euc-kr", na_values="*")
            print(file, ">>", e)
            problem_list.append(file)
            continue
        df_temp.rename(columns={'?"기준일ID"' : '기준일ID'})
        #결측값 0으로 대체
        df_temp.fillna(0, inplace=True)
        try:
            df_temp = create_dates(df_temp)
        except Exception as e:
            print(e)
        
        # 연령별로 컬럼 정리
        df_sum_per_day_집계구, df_sum_per_day_행정동 = combine_ages(df_temp)
        df_sum_per_day_집계구_list.append(df_sum_per_day_집계구)
        df_sum_per_day_행정동_list.append(df_sum_per_day_행정동)
    
    # 데이터 합치기
    df_concat_집계구 = pd.concat(df_sum_per_day_집계구_list)
    df_concat_행정동 = pd.concat(df_sum_per_day_행정동_list)

    return df_concat_집계구, df_concat_행정동

In [5]:
# 기준일ID 컬럼을 date형식으로 변환
# 년, 월 컬럼 생성
def create_dates(df_temp):
    df_temp.insert(0, '날짜', pd.to_datetime(df_temp['기준일ID'], format='%Y%m%d'))
    df_temp.insert(1, '년', df_temp['날짜'].dt.year)
    df_temp.insert(2, '월', df_temp['날짜'].dt.month)

    # 주말 여부 컬럼 생성
    df_temp.insert(3, "주말", df_temp['날짜'].dt.dayofweek > 4)

    # 기준일ID컬럼 drop
    df_temp.drop(columns=['기준일ID', '시간대구분'], inplace=True)
    # df_temp.drop(columns=['Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36','Unnamed: 37'], inplace=True)
    return df_temp

In [6]:
# 연령별로 컬럼 정리
def combine_ages(df_temp):
    cols = [
    '남자0세부터9세생활인구수', '여자0세부터9세생활인구수', 
    '남자10세부터14세생활인구수', '남자15세부터19세생활인구수', '여자10세부터14세생활인구수', '여자15세부터19세생활인구수', 
    '남자20세부터24세생활인구수', '남자25세부터29세생활인구수', '여자20세부터24세생활인구수', '여자25세부터29세생활인구수',
    '남자30세부터34세생활인구수', '남자35세부터39세생활인구수', '여자30세부터34세생활인구수', '여자35세부터39세생활인구수', 
    '남자40세부터44세생활인구수', '남자45세부터49세생활인구수', '여자40세부터44세생활인구수', '여자45세부터49세생활인구수', 
    '남자50세부터54세생활인구수', '남자55세부터59세생활인구수', '여자50세부터54세생활인구수', '여자55세부터59세생활인구수', 
    '남자60세부터64세생활인구수', '남자65세부터69세생활인구수', '여자60세부터64세생활인구수', '여자65세부터69세생활인구수', 
    '남자70세이상생활인구수', '여자70세이상생활인구수'
    ]
    df_temp['10세 이하'] = df_temp.남자0세부터9세생활인구수 + df_temp.여자0세부터9세생활인구수
    df_temp['10대'] = df_temp.남자10세부터14세생활인구수 + df_temp.남자15세부터19세생활인구수 + df_temp.여자10세부터14세생활인구수 + df_temp.여자15세부터19세생활인구수
    df_temp['20대'] = df_temp.남자20세부터24세생활인구수 + df_temp.남자25세부터29세생활인구수 + df_temp.여자20세부터24세생활인구수 + df_temp.여자25세부터29세생활인구수
    df_temp['30대'] = df_temp.남자30세부터34세생활인구수 + df_temp.남자35세부터39세생활인구수 + df_temp.여자30세부터34세생활인구수 + df_temp.여자35세부터39세생활인구수
    df_temp['40대'] = df_temp.남자40세부터44세생활인구수 + df_temp.남자45세부터49세생활인구수 + df_temp.여자40세부터44세생활인구수 + df_temp.여자45세부터49세생활인구수
    df_temp['50대'] = df_temp.남자50세부터54세생활인구수 + df_temp.남자55세부터59세생활인구수 + df_temp.여자50세부터54세생활인구수 + df_temp.여자55세부터59세생활인구수
    df_temp['60대'] = df_temp.남자60세부터64세생활인구수 + df_temp.남자65세부터69세생활인구수 + df_temp.여자60세부터64세생활인구수 + df_temp.여자65세부터69세생활인구수
    df_temp['70세 이상'] = df_temp.남자70세이상생활인구수 + df_temp.여자70세이상생활인구수
    df_temp.drop(columns=cols, inplace=True)
    df_sum_per_day_집계구 = df_temp.groupby(['날짜', '년', '월', '주말', '행정동코드', '집계구코드']).sum()
    df_sum_per_day_행정동 = df_temp.groupby(['날짜', '년', '월', '주말', '행정동코드']).sum()

    return df_sum_per_day_집계구, df_sum_per_day_행정동

In [7]:
#2019년
local_people_2019_df_집계구, local_people_2019_df_행정동 = data_process(local_people_2019_list, "./data/집계구 단위 서울 생활인구/local_people/2019/")

  0%|          | 0/351 [00:00<?, ?it/s]

In [ ]:
#2020년
local_people_2020_df_집계구, local_people_2020_df_행정동 = data_process(local_people_2020_list, "./data/집계구 단위 서울 생활인구/local_people/2020/")

In [ ]:
#2021년
local_people_2021_df_집계구, local_people_2021_df_행정동 = data_process(local_people_2021_list, "./data/집계구 단위 서울 생활인구/local_people/2021/")

In [ ]:
local_people_2019_df_집게구.to_csv("./data/집계구 단위 서울 생활인구/

In [20]:
local_people_2019_df_행정동

집계구코드     총생활인구수   10세 이하  \
날짜         년    월  주말    행정동코드                                            
2019-01-01 2019 1  False 11110515   685967863437823   372620.0  22194.0   
                         11110530   449229630797856   439819.0  21254.0   
                         11110540   132126481200360   112059.0   3323.0   
                         11110550   528506408882232   254159.0   9783.0   
                         11110560   977737746290664   434021.0  25476.0   
...                                             ...        ...      ...   
2019-12-31 2019 12 False 11740650  1296076050137736   664247.0  43889.0   
                         11740660  1269075593638392   655702.0  38460.0   
                         11740685  2565168764577696  1262952.0  78298.0   
                         11740690  1053065542888752   189565.0  11847.0   
                         11740700  1485093747161136   746423.0  48393.0   

                                       10대       20대       30대       40대  \
날짜         년    월  주말    행정동코드                                             
2019-01-01 2019 1  False 11110515  30171.0   49249.0   51156.0   66066.0   
                         11110530  25665.0   71920.0   72446.0   71744.0   
                         11110540   6905.0   17937.0   17625.0   18585.0   
                         11110550  19316.0   32744.0   31894.0   39249.0   
                         11110560  33919.0   41756.0   44645.0   62075.0   
...                                    ...       ...       ...       ...   
2019-12-31 2019 12 False 11740650  40479.0   87441.0  106373.0   99102.0   
                         11740660  55078.0   83131.0   97555.0  106364.0   
                         11740685  88735.0  156746.0  180616.0  192015.0   
                         11740690  21746.0   13876.0   18790.0   24928.0   
                         11740700  59746.0   82321.0   98240.0  104549.0   

                                        50대       60대    70세 이상  Unnamed: 33  \
날짜         년    월  주말    행정동코드                                                 
2019-01-01 2019 1  False 11110515   53370.0   37063.0   50367.0          NaN   
                         11110530   59943.0   46526.0   61449.0          NaN   
                         11110540   16383.0   14571.0   14962.0          NaN   
                         11110550   41315.0   31820.0   40194.0          NaN   
                         11110560   75199.0   58094.0   73375.0          NaN   
...                                     ...       ...       ...          ...   
2019-12-31 2019 12 False 11740650  103563.0   83348.0   78696.0          NaN   
                         11740660  102860.0   80058.0   71763.0          NaN   
                         11740685  202006.0  173472.0  149989.0          NaN   
                         11740690   22607.0   17957.0   23034.0          NaN   
                         11740700  110231.0  101613.0  115092.0          NaN   

                                   Unnamed: 34  Unnamed: 35  Unnamed: 36  \
날짜         년    월  주말    행정동코드                                             
2019-01-01 2019 1  False 11110515          NaN          NaN          NaN   
                         11110530          NaN          NaN          NaN   
                         11110540          NaN          NaN          NaN   
                         11110550          NaN          NaN          NaN   
                         11110560          NaN          NaN          NaN   
...                                        ...          ...          ...   
2019-12-31 2019 12 False 11740650          NaN          NaN          NaN   
                         11740660          NaN          NaN          NaN   
                         11740685          NaN          NaN          NaN   
                         11740690          NaN          NaN          NaN   
                         11740700          NaN          NaN          NaN   

                                   Unnamed: 37  
날짜

In [8]:
df = pd.read_csv("./data/집계구 단위 서울 생활인구/local_people/2019/local_people_20191030.csv")
df

,기준일ID,시간대구분,행정동코드,집계구코드,총생활인구수,남자0세부터9세생활인구수,남자10세부터14세생활인구수,남자15세부터19세생활인구수,남자20세부터24세생활인구수,남자25세부터29세생활인구수,...,여자50세부터54세생활인구수,여자55세부터59세생활인구수,여자60세부터64세생활인구수,여자65세부터69세생활인구수,여자70세이상생활인구수,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37
0,20191030,0,11110515,1101072010002,857,21,13,18,28,30,...,42,31,34,19,85,NaN,NaN,NaN,NaN,NaN
1,20191030,0,11110515,1101072010003,59,*,*,*,*,*,...,*,4,*,*,7,NaN,NaN,NaN,NaN,NaN
2,20191030,0,11110515,1101072010004,325,20,13,7,5,10,...,7,10,9,5,15,NaN,NaN,NaN,NaN,NaN
3,20191030,0,11110515,1101072010005,166,8,5,4,7,6,...,4,9,*,5,12,NaN,NaN,NaN,NaN,NaN
4,20191030,0,11110515,1101072010006,260,*,*,7,4,8,...,9,10,7,7,24,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459633,20191030,23,11740700,1125071022201,959,22,11,27,21,30,...,34,41,36,28,60,NaN,NaN,NaN,NaN,NaN
459634,20191030,23,11740700,1125071022202,233,4,*,6,5,7,...,8,9,8,6,14,NaN,NaN,NaN,NaN,NaN
459635,20191030,23,11740700,1125071022501,1246,55,27,27,17,29,...,37,76,53,49,91,NaN,NaN,NaN,NaN,NaN
459636,20191030,23,11740700,1125071022701,235,16,8,6,*,5,...,11,14,7,10,22,NaN,NaN,NaN,NaN,NaN


In [9]:
local_people_2019_df_행정동.groupby(['년', '월', '주말', '행정동코드']).mean()

집계구코드        총생활인구수        10세 이하  \
년    월  주말    행정동코드                                                
2019 1  False 11110515  6.848668e+14  3.933890e+05  22095.130435   
              11110530  4.492296e+14  7.767040e+05  20696.652174   
              11110540  1.321265e+14  1.575564e+05   3441.130435   
              11110550  5.283628e+14  2.642680e+05  11362.086957   
              11110560  9.775941e+14  4.303508e+05  26350.347826   
...                              ...           ...           ...   
     12 True  11740650  1.296076e+15  6.754988e+05  44050.000000   
              11740660  1.269076e+15  6.592179e+05  38433.000000   
              11740685  2.565169e+15  1.288847e+06  82882.888889   
              11740690  1.053066e+15  1.946696e+05  11625.777778   
              11740700  1.485094e+15  7.281070e+05  46524.111111   

                                 10대            20대            30대  \
년    월  주말    행정동코드                                                  
2019 1  False 11110515  31747.043478   55759.826087   62022.130435   
              11110530  27720.826087  123837.260870  173876.000000   
              11110540  10246.086957   24153.608696   29986.826087   
              11110550  23054.000000   35591.521739   31254.260870   
              11110560  38178.913043   42789.347826   42538.869565   
...                              ...            ...            ...   
     12 True  11740650  41765.555556   92166.555556  112311.777778   
              11740660  55628.000000   84222.000000  100720.111111   
              11740685  93527.000000  159932.777778  189504.000000   
              11740690  17732.111111   15055.111111   20827.444444   
              11740700  58911.222222   78963.111111   96110.000000   

                                  40대            50대            60대  \
년    월  주말    행정동코드                                                   
2019 1  False 11110515   73074.608696   54133.739130   35318.043478   
              11110530  165071.782609  114243.608696   68531.173913   
              11110540   28813.652174   24687.739130   16805.478261   
              11110550   38970.130435   42057.782609   32867.043478   
              11110560   58513.521739   70096.043478   58896.304348   
...                               ...            ...            ...   
     12 True  11740650  101932.000000  101464.888889   83326.666667   
              11740660  106994.444444  100688.555556   79970.111111   
              11740685  195211.555556  203193.666667  173226.222222   
              11740690   26976.333333   24678.000000   19163.222222   
              11740700  105475.555556  111181.444444   99653.444444   

                               70세 이상  Unnamed: 33  Unnamed: 34  Unnamed: 35  \
년    월  주말    행정동코드                                                            
2019 1  False 11110515   46147.565217          NaN          NaN          NaN   
              11110530   73894.608696          NaN          NaN          NaN   
              11110540   17634.043478          NaN          NaN          NaN   
              11110550   40904.434783          NaN          NaN          NaN   
              11110560   74300.086957          NaN          NaN          NaN   
...                               ...          ...          ...          ...   
     12 True  11740650   76850.333333          NaN          NaN          NaN   
              11740660   70730.777778          NaN          NaN          NaN   
              11740685  148016.000000          NaN          NaN          NaN   
              11740690   25157.333333          NaN          NaN          NaN   
              11740700  104963.333333          NaN          NaN          NaN   

                        Unnamed: 36  Unnamed: 37  
년    월  주말    행정동코드                               
2019 1  False 11110515          NaN          NaN  
              11110530          NaN          NaN  
              11110540          NaN          NaN  
              1

In [19]:
df_code = pd.read_excel("./data/집계구 단위 서울 생활인구/행정동코드_매핑정보_20200325.xlsx")
df_code


ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [ ]:
def name_dong(df):
    df_code = pd.read_excel("./data/집계구 단위 서울 생활인구/행정동코드_매핑정보_20200325.xlsx")
    df['행정동코드']=df['행정동코드'].astype(str)
    df_code['행자부행정동코드']=df_code['행자부행정동코드'].astype(str)
    df = pd.merge(df, df_code, how='inner',left_on='행정동코드', right_on='행자부행정동코드')


In [72]:
print(local_people_2019_df.info())

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 6722699 entries, (Timestamp('2019-01-01 00:00:00'), 2019, 1, False, 11110515, 1101072010001) to (Timestamp('2019-12-31 00:00:00'), 2019, 12, False, 11740700, 1125071023701)
Data columns (total 9 columns):
 #   Column  Dtype  
---  ------  -----  
 0   총생활인구수  float64
 1   10세 이하  float64
 2   10대     float64
 3   20대     float64
 4   30대     float64
 5   40대     float64
 6   50대     float64
 7   60대     float64
 8   70세 이상  float64
dtypes: float64(9)
memory usage: 520.0 MB
None


NameError: name 'local_people_2019_df_행정동' is not defined

In [38]:
local_people_2019_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 6722699 entries, (Timestamp('2019-01-01 00:00:00'), 2019, 1, False, 11110515, 1101072010001) to (Timestamp('2019-12-31 00:00:00'), 2019, 12, False, 11740700, 1125071023701)
Data columns (total 9 columns):
 #   Column  Dtype  
---  ------  -----  
 0   총생활인구수  float64
 1   10세 이하  float64
 2   10대     float64
 3   20대     float64
 4   30대     float64
 5   40대     float64
 6   50대     float64
 7   60대     float64
 8   70세 이상  float64
dtypes: float64(9)
memory usage: 520.0 MB


In [39]:
local_people_2019_df.shape

(6722699, 9)

In [89]:
df_local_people_2019_daily_avg = local_people_2019_df.groupby(['년', '월', '주말', '행정동코드', '집계구코드']).mean()

In [93]:
df_local_people_2019_daily_avg_dong = local_people_2019_df.groupby(['년', '월', '주말', '행정동코드']).sum()

In [94]:
df_local_people_2019_daily_avg_dong

총생활인구수       10세 이하          10대          20대  \
년    월  주말    행정동코드                                                           
2019 1  False 11110515  15130.344482   849.812709  1221.040134  2144.608696   
              11110530  45688.470588  1217.450128  1630.636829  7284.544757   
              11110540  31511.286957   688.226087  2049.217391  4830.721739   
              11110550  13213.400000   568.104348  1152.700000  1779.576087   
              11110560  11631.103408   712.171563  1031.862515  1156.468860   
...                              ...          ...          ...          ...   
     12 True  11740650  14072.891204   917.708333   870.115741  1920.136574   
              11740660  14025.912530   817.723404  1183.574468  1791.957447   
              11740685  13566.807018   872.451462   984.494737  1683.502924   
              11740690   4991.527066   298.096866   454.669516   386.028490   
              11740700  13238.309091   845.892929  1071.113131  1435.692929   

                                 30대          40대          50대          60대  \
년    월  주말    행정동코드                                                           
2019 1  False 11110515   2385.466555  2810.561873  2082.066890  1358.386288   
              11110530  10228.000000  9710.104859  6720.212276  4031.245524   
              11110540   5997.365217  5762.730435  4937.547826  3361.095652   
              11110550   1562.713043  1948.506522  2102.889130  1643.352174   
              11110560   1149.699177  1581.446533  1894.487662  1591.792009   
...                              ...          ...          ...          ...   
     12 True  11740650   2339.828704  2123.583333  2113.851852  1735.972222   
              11740660   2142.981087  2276.477541  2142.309693  1701.491726   
              11740685   1994.778947  2054.858480  2138.880702  1823.433918   
              11740690    534.037037   691.700855   632.769231   491.364672   
              11740700   1747.454545  1917.737374  2021.480808  1811.880808   

                             70세 이상  
년    월  주말    행정동코드                  
2019 1  False 11110515  1774.906355  
              11110530  4346.741688  
              11110540  3526.808696  
              11110550  2045.221739  
              11110560  2008.110458  
...                             ...  
     12 True  11740650  1601.048611  
              11740660  1504.910165  
              11740685  1558.063158  
              11740690   645.059829  
              11740700  1908.424242  

[10176 rows x 9 columns]

In [95]:
df_local_people_2019_daily_avg_dong.loc[(2019, 1, True, 11110515)]

총생활인구수    14757.649038
10세 이하      841.413462
10대        1193.668269
20대        2187.235577
30대        2116.817308
40대        2654.086538
50대        2033.091346
60대        1393.456731
70세 이상     1862.379808
Name: (2019, 1, True, 11110515), dtype: float64

In [96]:
df_local_people_2019_daily_avg_dong.loc[(2019, 1, False, 11110515)]

총생활인구수    15130.344482
10세 이하      849.812709
10대        1221.040134
20대        2144.608696
30대        2385.466555
40대        2810.561873
50대        2082.066890
60대        1358.386288
70세 이상     1774.906355
Name: (2019, 1, False, 11110515), dtype: float64

In [92]:
df_local_people_2019_daily_avg.loc[(2019, 1, True, 11110515)]

,총생활인구수,10세 이하,10대,20대,30대,40대,50대,60대,70세 이상
집계구코드,,,,,,,,,
1101072010001,6807.750,455.750,591.625,1003.875,935.000,1144.625,908.125,460.750,808.500
1101072010002,17123.750,875.250,1390.000,2828.625,2655.750,3060.250,2291.250,1626.500,2073.250
1101072010003,6124.375,364.750,455.500,852.875,794.875,1076.250,898.875,499.375,660.750
1101072010004,8370.000,588.125,756.625,1161.250,1166.625,1372.000,1044.750,732.750,1014.125
1101072010005,5853.000,388.750,469.375,803.750,804.750,1047.500,815.250,345.375,657.875
1101072010006,3160.750,187.250,178.500,189.250,365.000,600.500,267.375,205.500,345.625
1101072010007,6410.875,469.250,552.875,724.250,876.875,1211.750,744.250,592.625,797.625
1101072010008,8507.250,610.750,768.875,1227.125,1206.000,1531.125,1184.875,527.875,986.375
1101072010009,48116.125,1995.875,3460.375,8955.000,7842.250,8278.875,6380.500,4700.500,6176.375


In [87]:
local_people_2019_df_month.loc[(2019, 1, False, 11110515)]

,총생활인구수,10세 이하,10대,20대,30대,40대,50대,60대,70세 이상
집계구코드,,,,,,,,,
1101072010001,7263.086957,509.826087,638.956522,1075.521739,1074.826087,1282.086957,965.869565,491.130435,757.608696
1101072010002,17644.826087,931.565217,1494.478261,2579.217391,2820.347826,3162.130435,2357.130435,1726.869565,2247.173913
1101072010003,6829.304348,455.260870,639.434783,881.695652,974.739130,1207.304348,934.043478,552.304348,709.347826
1101072010004,6611.000000,488.739130,587.652174,961.956522,929.478261,1052.217391,837.086957,554.695652,704.043478
1101072010005,5836.652174,422.391304,543.739130,767.000000,814.608696,1059.086957,769.782609,322.478261,577.391304
1101072010006,3255.086957,195.956522,168.782609,190.565217,354.391304,549.434783,271.565217,194.782609,383.739130
1101072010007,5247.304348,391.695652,442.956522,618.347826,695.391304,892.217391,614.000000,425.782609,593.956522
1101072010008,10947.043478,841.043478,1097.521739,1652.608696,1721.652174,2050.043478,1459.130435,677.869565,1056.086957
1101072010009,48973.782609,2020.217391,3423.521739,8256.869565,8716.869565,9050.739130,6793.130435,4565.391304,5823.956522


In [88]:
local_people_2019_df_month.loc[(2019, 1, True, 11110515)]

,총생활인구수,10세 이하,10대,20대,30대,40대,50대,60대,70세 이상
집계구코드,,,,,,,,,
1101072010001,6807.750,455.750,591.625,1003.875,935.000,1144.625,908.125,460.750,808.500
1101072010002,17123.750,875.250,1390.000,2828.625,2655.750,3060.250,2291.250,1626.500,2073.250
1101072010003,6124.375,364.750,455.500,852.875,794.875,1076.250,898.875,499.375,660.750
1101072010004,8370.000,588.125,756.625,1161.250,1166.625,1372.000,1044.750,732.750,1014.125
1101072010005,5853.000,388.750,469.375,803.750,804.750,1047.500,815.250,345.375,657.875
1101072010006,3160.750,187.250,178.500,189.250,365.000,600.500,267.375,205.500,345.625
1101072010007,6410.875,469.250,552.875,724.250,876.875,1211.750,744.250,592.625,797.625
1101072010008,8507.250,610.750,768.875,1227.125,1206.000,1531.125,1184.875,527.875,986.375
1101072010009,48116.125,1995.875,3460.375,8955.000,7842.250,8278.875,6380.500,4700.500,6176.375


In [9]:
# local_people_2019_df = concat_files(local_people_2019_list, "./data/집계동 단위 서울 생활인구/local_people/2019/")
# local_people_2020_df = concat_files(local_people_2020_list, "./data/집계동 단위 서울 생활인구/local_people/2020/")
# local_people_2021_df = concat_files(local_people_2021_list, "./data/집계동 단위 서울 생활인구/local_people/2021/")


# long_foreigner_df = concat_files(long_foreigner_list, "./data/집계동 단위 서울 생활인구/long_foreigner/")
# temp_foreinger_df = concat_files(temp_foreinger_list, "./data/집계동 단위 서울 생활인구/temp_foreigner/")
# print("local_people_df.shape >>", local_people_df.shape)
# print("long_foreigner_df.shape >>", long_foreigner_df.shape)
# print("temp_foreinger_df.shape >>", temp_foreinger_df.shape)

In [10]:
# # 결측값 0으로 대체
# local_people_df.fillna(0, inplace=True)
# local_people_df.info()

In [11]:
# for col in tqdm(cols):
#     if col

In [12]:
# local_people_df['10대 남성'] = local_people_df.남자10세부터14세생활인구수 + local_people_df.남자15세부터19세생활인구수
# local_people_df['20대 남성'] = 